# Web Scraping LinkedIn

In [ ]:
#!pip install selenium

In [ ]:
import pandas as pd
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
browser = webdriver.Chrome("chromedriver.exe")
browser.get("https://www.linkedin.com/login/")

In [ ]:
username = browser.find_element(By.ID, "username")
username.send_keys("masukkan email")

password = browser.find_element(By.ID, "password")
password.send_keys("masukkan password")

time.sleep(2)

login_button = browser.find_element(By.XPATH, "//button[@class = 'btn__primary--large from__button--floating']")
login_button.click()

### Scrap Multiple Page -> Untuk  Search Term List (Beberapa Kata Kunci)

In [ ]:
# FUNCTION UNTUK MENCARI DARI SEARCH TERM LIST

def job_linkedin_list(search_term_list, search_loc):
    # Data frame awal
    df = pd.DataFrame(columns = ["job_name", "company", "lokasi", "icon_keterangan"])
        
    # For looping untuk "search term list"
    for search_term in search_term_list:
        
        browser.get("https://www.linkedin.com/jobs/")
        time.sleep(1)
        
        # Isian untuk cari berdasarkan pekerjaan dan lokasi
        search_term_isi = browser.find_element(By.XPATH, "//input[starts-with (@id, 'jobs-search-box-keyword-id-')]")
        search_term_isi.send_keys(search_term)

        time.sleep(1)

        search_loc_isi = browser.find_element(By.XPATH, "//input[starts-with (@id, 'jobs-search-box-location-id-')]")
        search_loc_isi.send_keys(search_loc)

        time.sleep(1)

        search_loc_isi.send_keys(Keys.RETURN)
        
        #-------------------------------------------------------------------------------
        # UNTUK PAGE = 1 (langsung scraping saja karena tidak perlu klik page button-nya)
        # Mengecilkan ukuran window
        browser.set_window_size(600, 1080)

        # Scroll sampai bawah dan kembalikan ke atas
        element = browser.find_element(By.TAG_NAME, "body")
        for i in range(1, 7): # For looping untuk scroll down 6x
            browser.execute_script("window.scrollBy(0, 560)", "")
            time.sleep(0.5)
        time.sleep(1)
        element.send_keys(Keys.HOME)

        # Memperbesar kembali ukuran window 
        browser.maximize_window()

        # List nama-nama lowongan
        job_all = browser.find_elements(By.XPATH, "//a[@class = 'disabled ember-view job-card-container__link job-card-list__title']")
        list_job_oke = [i for i in job_all if search_term in i.text.lower()]

        # For looping untuk "list job oke" (job yang sesuai dengan search term)
        for job in list_job_oke:
            job.click()
            time.sleep(1)

            job_name = browser.find_element(By.XPATH, "//h2[@class = 't-16 t-black t-bold truncate']").text
            company = browser.find_element(By.XPATH, "//span[@class = 'jobs-unified-top-card__subtitle-primary-grouping mr2 t-black']/span").text
            lokasi = browser.find_element(By.XPATH, "//span[@class = 'jobs-unified-top-card__bullet']").text

            icon = browser.find_elements(By.XPATH, "//div[starts-with (@class, 'ivm-image-view-model')]/div/li-icon")
            list_icon = [i.get_attribute("type") for i in icon]
            if "lightbulb" in list_icon:
                list_icon.remove("lightbulb")

            keterangan = browser.find_elements(By.XPATH, "//div[@class = 'jobs-unified-top-card__job-insight']")
            list_keterangan = [k.text for k in keterangan]

            icon_keterangan = dict(zip(list_icon, list_keterangan))

            df = df.append({"job_name": job_name, "company": company, "lokasi": lokasi, 
                            "icon_keterangan": icon_keterangan}, ignore_index = True)

        #-------------------------------------------------------------------------------
        # UNTUK PAGE > 1 (perlu klik page button-nya)
        max_pages = [int(num_page.text) for num_page in browser.find_elements(By.XPATH, "//li[@data-test-pagination-page-btn]")][-1]
        pages = [x for x in range(2, max_pages+1)]

        # For looping untuk page (agar bisa ke next page)
        for page in pages:
            if page < 9 or page > 9:
                page_button = browser.find_element(By.XPATH, "//li[@data-test-pagination-page-btn =" + str(page) + "]")
            elif page == 9:
                page_button = browser.find_elements(By.XPATH, "//li[@class = 'artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view']")[7]

            page_button.click()
            time.sleep(2)

            # Mengecilkan ukuran window
            browser.set_window_size(600, 1080)

            # Scroll sampai bawah dan kembalikan ke atas
            element = browser.find_element(By.TAG_NAME, "body")
            for i in range(1, 7): # For looping untuk scroll down 6x
                browser.execute_script("window.scrollBy(0, 560)", "")
                time.sleep(0.5)

            time.sleep(1)
            element.send_keys(Keys.HOME)

            # Memperbesar kembali ukuran window 
            browser.maximize_window()

            job_all = browser.find_elements(By.XPATH, "//a[@class = 'disabled ember-view job-card-container__link job-card-list__title']")
            list_job_oke = [i for i in job_all if search_term in i.text.lower()]

            # For looping untuk "list job oke" (job yang sesuai dengan search term)
            for job in list_job_oke:
                job.click()
                time.sleep(1)

                job_name = browser.find_element(By.XPATH, "//h2[@class = 't-16 t-black t-bold truncate']").text
                company = browser.find_element(By.XPATH, "//span[@class = 'jobs-unified-top-card__subtitle-primary-grouping mr2 t-black']/span").text
                lokasi = browser.find_element(By.XPATH, "//span[@class = 'jobs-unified-top-card__bullet']").text

                icon = browser.find_elements(By.XPATH, "//div[starts-with (@class, 'ivm-image-view-model')]/div/li-icon")
                list_icon = [i.get_attribute("type") for i in icon]
                if "lightbulb" in list_icon:
                    list_icon.remove("lightbulb")

                keterangan = browser.find_elements(By.XPATH, "//div[@class = 'jobs-unified-top-card__job-insight']")
                list_keterangan = [k.text for k in keterangan]

                icon_keterangan = dict(zip(list_icon, list_keterangan))

                df = df.append({"job_name": job_name, "company": company, "lokasi": lokasi, 
                                "icon_keterangan": icon_keterangan}, ignore_index = True)

    return df

In [ ]:
start_time = datetime.now()

search_term_list = ["data scientist", "data analyst", "data engineer"]
hasil = job_linkedin_list(search_term_list, "indonesia")

end_time = datetime.now()
print("Durasi Scraping: {}".format(end_time - start_time))

In [ ]:
len(hasil)

In [ ]:
# Save to csv
hasil.to_csv("Scraping Profesi Data (Linkedin).csv", index = False)

# Data diambil tanggal 4 Nov 2021

In [ ]:
browser.close()

In [ ]:
# Membaca data yang sudah disimpan
df = pd.read_csv("Scraping Profesi Data (Linkedin).csv")
df.head()